In [28]:
import math
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

In [29]:
from loguru import logger
from 有用なスクリプト import load_吾輩は猫である
for sentence in load_吾輩は猫である(path="./吾輩は猫であるutf8.txt")[:10]:
    print(sentence if len(sentence) < 50 else sentence[:50]+"　（以下略）")

吾輩は猫である

夏目漱石



　吾輩《わがはい》は猫である。名前はまだ無い。

　どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事　（以下略）
　この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め　（以下略）
　ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじん》の母　（以下略）
　ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考え　（以下略）
　吾輩の主人は滅多《めった》に吾輩と顔を合せる事がない。職業は教師だそうだ。学校から帰ると終日書斎に　（以下略）
　吾輩がこの家へ住み込んだ当時は、主人以外のものにははなはだ不人望であった。どこへ行っても跳《は》ね　（以下略）


In [37]:
import math
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

# VOCAB_SIZE = 5000 # 辞書の大きさ
VOCAB_SIZE = 5000 # 辞書の大きさ

# 初期化
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder = decoders.ByteLevel()

# Initialize a trainer
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE)

# Train the tokenizer
print(f"Training BPE tokenizer with vocab size {VOCAB_SIZE}...")
tokenizer.train(files=["./吾輩は猫であるutf8.txt"], trainer=trainer)
print("Training complete.")

# --- 3. Target Text to Compress ---
text_to_compress = "吾輩わがはいは猫である。名前はまだ無い。"

print(f"\n--- Comparing Bits for Text ---")
print(f"Target Text: '{text_to_compress}'")

# --- 4. Calculate 'Unicode' (UTF-8) Bits ---
# We encode the string into bytes using UTF-8
utf8_bytes = text_to_compress.encode('utf-8')
utf8_bit_count = len(utf8_bytes) * 8

print(f"\n[UTF-8 Calculation]")
print(f"  Total bytes: {len(utf8_bytes)}")
print(f"  Total bits (bytes * 8): {utf8_bit_count}")

# --- 5. Calculate BPE Bits ---
# Encode the text using our trained BPE model
encoding = tokenizer.encode(text_to_compress)
bpe_tokens = encoding.ids

# The number of bits per token is determined by the vocabulary size.
# We need log2(vocab_size) bits to represent any token ID.
actual_vocab_size = tokenizer.get_vocab_size()
bits_per_token = math.ceil(math.log2(actual_vocab_size)) 

# Total bits = number of tokens * bits per token
total_bpe_bits = len(bpe_tokens) * bits_per_token

print(f"\n[BPE Calculation]")
print(f"  Vocabulary size: {actual_vocab_size}")
print(f"  Bits needed per token (ceil(log2(vocab_size))): {bits_per_token}")
print(f"  Number of tokens: {len(bpe_tokens)}")
print(f"  BPE Tokens: {bpe_tokens}")
print(f"  Total bits (tokens * bits_per_token): {total_bpe_bits}")

# --- 6. Final Comparison ---
print("\n--- 📊 Final Result ---")
print(f"UTF-8 Bits: {utf8_bit_count}")
print(f"BPE Bits:   {total_bpe_bits}")

compression_ratio = utf8_bit_count / total_bpe_bits
print(f"\nCompression Ratio: {compression_ratio:.2f} (UTF-8 bits / BPE bits)")

Training BPE tokenizer with vocab size 5000...
Training complete.

--- Comparing Bits for Text ---
Target Text: '吾輩わがはいは猫である。名前はまだ無い。'

[UTF-8 Calculation]
  Total bytes: 60
  Total bits (bytes * 8): 480

[BPE Calculation]
  Vocabulary size: 5000
  Bits needed per token (ceil(log2(vocab_size))): 13
  Number of tokens: 10
  BPE Tokens: [315, 2459, 535, 144, 4662, 142, 1873, 3784, 3646, 142]
  Total bits (tokens * bits_per_token): 130

--- 📊 Final Result ---
UTF-8 Bits: 480
BPE Bits:   130

Compression Ratio: 3.69 (UTF-8 bits / BPE bits)


In [31]:
tokenizer.decode(encoding.ids)

'吾輩わがはいは猫である。名前はまだ無い。'